<h1 align="center">Rich's Week 3 Project Notebook</h1>
<h2 align="center">Coursera IBM Data Science Professional Certificate</h2>
This notebook will primarily be used to fulfill the requirements of the Coursera IBM Data Science Capstone project.

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


<h2>Rich Huebner</h2>
<h3>Week 3 Project Assignment - Neighborhoods in Toronto</h3>

In [3]:
from pandas.io.json import json_normalize
import folium
from geopy.geocoders import Nominatim 
import geopy
import requests
from urllib.request import urlopen
from sklearn.cluster import KMeans
from IPython.display import display_html

print('Libraries are now included.')

Libraries are now included.


In [4]:
# Calls to the Foursquare API
# https://api.foursquare.com/v2/search/
# include ClientID, Client Secret, Version.
CLIENT_ID = '0PNU1Y1L1N0W33QOPPR0JMJYJRASC2NNJSCH5JPXXFEO2BIY' # your Foursquare ID
CLIENT_SECRET = '2LX3Z3HXAEKA1DZTT54TQDE4ZMSCYSLAZGKGZSK5ZIQ4IYMP' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('    Client ID: ' + CLIENT_ID)
print('Client Secret: ' + CLIENT_SECRET)

    Client ID: 0PNU1Y1L1N0W33QOPPR0JMJYJRASC2NNJSCH5JPXXFEO2BIY
Client Secret: 2LX3Z3HXAEKA1DZTT54TQDE4ZMSCYSLAZGKGZSK5ZIQ4IYMP


In [5]:
# Clean, straightforward way to grab the table using pandas read_html command. 
# This method seems to work well.
toronto_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto_data = pd.read_html(toronto_url)
df = toronto_data[0]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [6]:
# Create a new df where Borough != 'Not assigned'
df_na = df[df.Borough != 'Not assigned']
df_na = df_na.sort_values(by = ['Postal Code', 'Borough'])

# Remove space from the name of the attribute, so we can merge this later - basically renaming the columns.
df_na.columns = ['Postcode', 'Borough', 'Neighborhood']
df_na

,Postcode,Borough,Neighborhood
9,M1B,Scarborough,"Malvern, Rouge"
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
36,M1G,Scarborough,Woburn
45,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
107,M9P,Etobicoke,Westmount
116,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
143,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [7]:
df_na.shape

(103, 3)

<h2>Part II:  Geographic Coordinates - Lat/Long</h2>

In [8]:
# Going to yank these from a csv file.
geourl = "https://cocl.us/Geospatial_data"
geodata = pd.read_csv(geourl)
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
# Going to rename the columns for ease of use
geodata.columns = ['Postcode', 'lat', 'lon']
# and merge the data on Postcode.
toronto_data = pd.merge(df_na, geodata, on = 'Postcode')

In [10]:
# Let's look at the combined dataset now. I like it!
# Also do a little more renaming for ease of analysis later on.
toronto_data.columns = ['postcode', 'borough', 'nbh', 'lat', 'lon']
toronto_data


,postcode,borough,nbh,lat,lon
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


<h2>Part III:  Clustering and Mapping the Toronto Boroughs </h2>

<h4>In this first section, I'll focus on playing around with mapping and exploring the different venues.
</h4>

In [11]:
# First, let's make sure we have Toronto on the map - of where the center of it is -- for folium map.
address = 'Toronto, Ontario'
geolocator = geopy.Nominatim(user_agent="ny_explorer", timeout=30)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical center of Toronto, Ontario Canada is:  {}, {}.'.format(latitude, longitude))


The geograpical center of Toronto, Ontario Canada is:  43.6534817, -79.3839347.


In [12]:
# Check to make sure we have a folium map of Toronto to begin with.
tmap = folium.Map(location = [latitude, longitude], zoom_start = 13)
tmap

In [13]:
td = toronto_data
td.head()

,postcode,borough,nbh,lat,lon
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
# Next, for this part, let's fill in the various neighborhoods.

for lat, lng, nb in zip(td['lat'], td['lon'], td['nbh']):
    label = '{}'.format(nb)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#112233',
        fill_opacity=0.2,
        parse_html=False).add_to(tmap)  
    
tmap

    

In [15]:
# Define limits of results and radius (in meters)
LIMIT = 100
radius = 500
for n, lat, long in zip(td.nbh, td.lat, td.lon):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    long, 
    radius, 
    LIMIT)
    
    data = requests.get(url).json
    
tmap
    

In [16]:
# Next, let's look at JUST the boroughs that have Toronto in them.
df3 = td.copy()
df3 = df3[td.borough.str.contains("Toronto")].reset_index(drop=True)
df3.head()


,postcode,borough,nbh,lat,lon
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [17]:
# Next, we're only going to map these ones that contain Toronto.
tmap2 = folium.Map(location = [latitude, longitude], zoom_start = 13)
for lat, lng, nb in zip(df3['lat'], df3['lon'], df3['nbh']):
    label = '{}'.format(nb)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#112233',
        fill_opacity=0.2,
        parse_html=False).add_to(tmap2)  
    
tmap2


In [18]:
# Toronto (or boroughs containing Toronto) is assigned to df3 now.
# Get the neihborhood's name. What is the first neighborhood to work with?

# Set the location index
LOC_INDEX = 10

df3.loc[LOC_INDEX, 'nbh']

'Rosedale'

In [19]:
# Here's all the data I have on "The Beaches"
df3.iloc[LOC_INDEX]

postcode                 M4W
borough     Downtown Toronto
nbh                 Rosedale
lat                  43.6796
lon                 -79.3775
Name: 10, dtype: object

In [20]:
# It looks like this first neighborhood is "The Beaches". We'll run our analysis from here.
# Grab the neighborhood lat and long values.

neighborhood_latitude = df3.loc[LOC_INDEX, 'lat'] # neighborhood latitude value
neighborhood_longitude = df3.loc[LOC_INDEX, 'lon'] # neighborhood longitude value
neighborhood_name = df3.loc[LOC_INDEX, 'nbh'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


In [21]:
# Next, get top 100 venues that are in "The Beaches" within a radius of 500 meters.
LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=0PNU1Y1L1N0W33QOPPR0JMJYJRASC2NNJSCH5JPXXFEO2BIY&client_secret=2LX3Z3HXAEKA1DZTT54TQDE4ZMSCYSLAZGKGZSK5ZIQ4IYMP&v=20180604&ll=43.6795626,-79.37752940000001&radius=500&limit=50'

In [22]:
# Grab the results from the Foursquare API
results = requests.get(url).json()["response"]['groups'][0]['items']
results


[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4aff2d47f964a520743522e3',
   'name': 'Rosedale Park',
   'location': {'address': '38 Scholfield Ave.',
    'crossStreet': 'at Edgar Ave.',
    'lat': 43.68232820227814,
    'lng': -79.37893434347683,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.68232820227814,
      'lng': -79.37893434347683}],
    'distance': 327,
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['38 Scholfield Ave. (at Edgar Ave.)',
     'Toronto ON',
     'Canada']},
   'categories': [{'id': '4bf58dd8d48988d1e7941735',
     'name': 'Playground',
     'pluralName': 'Playgrounds',
     'shortName': 'Playground',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/playground_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'gro

In [23]:
# Try to read the json information
import json
venues_nbh = pd.json_normalize(results)
venues_nbh

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups
0,e-0-4aff2d47f964a520743522e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4aff2d47f964a520743522e3,Rosedale Park,38 Scholfield Ave.,at Edgar Ave.,43.682328,-79.378934,"[{'label': 'display', 'lat': 43.68232820227814...",327,CA,Toronto,ON,Canada,"[38 Scholfield Ave. (at Edgar Ave.), Toronto O...","[{'id': '4bf58dd8d48988d1e7941735', 'name': 'P...",0,[]
1,e-0-4bd777aa5cf276b054639b00-1,0,"[{'summary': 'This spot is popular', 'type': '...",4bd777aa5cf276b054639b00,Whitney Park,NaN,NaN,43.682036,-79.373788,"[{'label': 'display', 'lat': 43.68203573063681...",408,CA,NaN,NaN,Canada,[Canada],"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[]
2,e-0-501fc63de4b099ff781066c7-2,0,"[{'summary': 'This spot is popular', 'type': '...",501fc63de4b099ff781066c7,Rosedale Tennis Club,NaN,NaN,43.683226,-79.378984,"[{'label': 'display', 'lat': 43.6832255981443,...",424,CA,NaN,NaN,Canada,[Canada],"[{'id': '4e39a956bd410d7aed40cbc3', 'name': 'T...",0,[]
3,e-0-4d0e77df76cc37045715767c-3,0,"[{'summary': 'This spot is popular', 'type': '...",4d0e77df76cc37045715767c,Alex Murray Parkette,107 Crescent Road,South Drive,43.678300,-79.382773,"[{'label': 'display', 'lat': 43.67830024047895...",444,CA,Toronto,ON,Canada,"[107 Crescent Road (South Drive), Toronto ON, ...","[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",0,[]
4,e-0-4ef8f2a3775b54cdb5bdec7c-4,0,"[{'summary': 'This spot is popular', 'type': '...",4ef8f2a3775b54cdb5bdec7c,Milkman's Lane,South Dr,at Glen Rd,43.676352,-79.373842,"[{'label': 'display', 'lat': 43.67635206801555...",464,CA,Toronto,ON,Canada,"[South Dr (at Glen Rd), Toronto ON, Canada]","[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",0,[]


In [24]:
# Feel like we need to try another approach....

In [25]:
# Loops through the results list and places it in a pandas data frame for ease of access
# Create an empty list for the venues.
import requests
LIMIT = 20
my_radius = 500
def get_venues(names, latitudes, longitudes, radius=my_radius):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
 
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [26]:
venues = get_venues(names=df3['nbh'], latitudes=df3['lat'], longitudes=df3['lon'], radius = 500)
venues

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Rosedale Park,43.682328,-79.378934,Playground
1,The Beaches,43.676357,-79.293031,Whitney Park,43.682036,-79.373788,Park
2,The Beaches,43.676357,-79.293031,Rosedale Tennis Club,43.683226,-79.378984,Tennis Court
3,The Beaches,43.676357,-79.293031,Alex Murray Parkette,43.678300,-79.382773,Park
4,The Beaches,43.676357,-79.293031,Milkman's Lane,43.676352,-79.373842,Trail
...,...,...,...,...,...,...,...
190,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Rosedale Park,43.682328,-79.378934,Playground
191,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Whitney Park,43.682036,-79.373788,Park
192,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Rosedale Tennis Club,43.683226,-79.378984,Tennis Court
193,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Alex Murray Parkette,43.678300,-79.382773,Park


In [27]:
venues
# The data frame is much cleaner this way.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Rosedale Park,43.682328,-79.378934,Playground
1,The Beaches,43.676357,-79.293031,Whitney Park,43.682036,-79.373788,Park
2,The Beaches,43.676357,-79.293031,Rosedale Tennis Club,43.683226,-79.378984,Tennis Court
3,The Beaches,43.676357,-79.293031,Alex Murray Parkette,43.678300,-79.382773,Park
4,The Beaches,43.676357,-79.293031,Milkman's Lane,43.676352,-79.373842,Trail
...,...,...,...,...,...,...,...
190,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Rosedale Park,43.682328,-79.378934,Playground
191,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Whitney Park,43.682036,-79.373788,Park
192,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Rosedale Tennis Club,43.683226,-79.378984,Tennis Court
193,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Alex Murray Parkette,43.678300,-79.382773,Park


In [28]:
venues.shape

(195, 7)

In [29]:
# I think I want to just explore just the venues in "Rosedale" neighborhood.
venues2 = venues[venues['Neighborhood'] == 'Rosedale']
venues2

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
50,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
51,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
52,Rosedale,43.679563,-79.377529,Rosedale Tennis Club,43.683226,-79.378984,Tennis Court
53,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
54,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail


In [30]:
# Next, I'm going to map all the venues in Rosedale.
# Rosedale lat / long
latitude = 43.679563
longitude = -79.377529

tmap3 = folium.Map(location = [latitude, longitude], zoom_start = 15)
# Let's place a circle marker on the neighborhood center.
folium.CircleMarker([latitude, longitude], radius = 5, popup="Rosedale", color='Red').add_to(tmap3)

for lat, lng, v, cat in zip(venues2['Venue Latitude'], venues2['Venue Longitude'], venues2['Venue'], venues2['Venue Category']):
    label = '{}'.format(v)
    label = folium.Popup(label, parse_html=True)
    folium.Marker([lat, lng], popup=label, icon=folium.Icon(color = 'blue')).add_to(tmap3)
    
    #folium.CircleMarker(
    #    [lat, lng],
    #    radius=10,
    #    popup=label,
    #    color='blue',
    #    fill=True,
    #    fill_color='#112233',
    #    fill_opacity=0.2,
    #    parse_html=False).add_to(tmap3)  
    
tmap3


In [31]:
# Let's try the clustering at this point.
td

,postcode,borough,nbh,lat,lon
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [32]:
td.head()

,postcode,borough,nbh,lat,lon
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [33]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(td['lat'], td['lon'], td['postcode'], td['borough'], td['nbh']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [34]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['postcode', 'borough', 'nbh', 'boroughlatitude', 'boroughlongitude', 'venuename', 
                     'venuelatitude', 'venuelongitude', 'venuecategory']
venues_df

,postcode,borough,nbh,boroughlatitude,boroughlongitude,venuename,venuelatitude,venuelongitude,venuecategory
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
...,...,...,...,...,...,...,...,...,...
2162,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,LCBO,43.741508,-79.584501,Liquor Store
2163,M9W,Etobicoke,"Northwest, West Humber - Clairville",43.706748,-79.594054,Economy Rent A Car,43.708471,-79.589943,Rental Car Location
2164,M9W,Etobicoke,"Northwest, West Humber - Clairville",43.706748,-79.594054,Logistics Distribution,43.707554,-79.589252,Bar
2165,M9W,Etobicoke,"Northwest, West Humber - Clairville",43.706748,-79.594054,Saand Rexdale,43.705072,-79.598725,Drugstore


In [35]:
# How many vanues are there in each post code?
venues_df.groupby(["postcode", "borough", "nbh"]).count()

,,,boroughlatitude,boroughlongitude,venuename,venuelatitude,venuelongitude,venuecategory
postcode,borough,nbh,,,,,,
M1B,Scarborough,"Malvern, Rouge",1,1,1,1,1,1
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",2,2,2,2,2,2
M1E,Scarborough,"Guildwood, Morningside, West Hill",8,8,8,8,8,8
M1G,Scarborough,Woburn,4,4,4,4,4,4
M1H,Scarborough,Cedarbrae,8,8,8,8,8,8
...,...,...,...,...,...,...,...,...
M9N,York,Weston,2,2,2,2,2,2
M9P,Etobicoke,Westmount,6,6,6,6,6,6
M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens",4,4,4,4,4,4


In [36]:
# What are the different categories we're working with???
venues_df['venuecategory'].unique()[:50]

array(['Fast Food Restaurant', 'Bar', 'Construction & Landscaping',
       'Bank', 'Electronics Store', 'Restaurant', 'Mexican Restaurant',
       'Rental Car Location', 'Medical Center', 'Intersection',
       'Breakfast Spot', 'Coffee Shop', 'Korean BBQ Restaurant',
       'Hakka Restaurant', 'Caribbean Restaurant', 'Thai Restaurant',
       'Athletics & Sports', 'Gas Station', 'Bakery',
       'Fried Chicken Joint', 'Playground', 'Jewelry Store', 'Smoke Shop',
       'Department Store', 'Hobby Shop', 'Train Station',
       'Ice Cream Shop', 'Bus Line', 'Metro Station', 'Bus Station',
       'Park', 'Soccer Field', 'Motel', 'American Restaurant', 'Café',
       'General Entertainment', 'Skating Rink', 'College Stadium',
       'Chinese Restaurant', 'Indian Restaurant', 'Vietnamese Restaurant',
       'Pet Store', 'Sandwich Place', 'Middle Eastern Restaurant',
       'Auto Garage', 'Accessories Store', 'Latin American Restaurant',
       'Lounge', 'Clothing Store', 'Italian Restauran

In [37]:
# In this section, I use the one hot encoder to make a column for each of the various categories.
toronto_onehot = pd.get_dummies(venues_df[['venuecategory']], prefix="", prefix_sep="")
# add postal, borough and neighborhood column back to dataframe
toronto_onehot['postcode'] = venues_df['postcode'] 
toronto_onehot['borough'] = venues_df['borough'] 
toronto_onehot['nbh'] = venues_df['nbh'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(2167, 280)


,postcode,borough,nbh,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,Scarborough,"Malvern, Rouge",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1E,Scarborough,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
toronto_grouped = toronto_onehot.groupby(["postcode", "borough", "nbh"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(100, 280)


,postcode,borough,nbh,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,Scarborough,"Malvern, Rouge",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M1G,Scarborough,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,Scarborough,Cedarbrae,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,M9N,York,Weston,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,M9P,Etobicoke,Westmount,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
# Let's check out the top 5 venues.
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['postcode', 'borough', 'nbh']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['postcode'] = toronto_grouped['postcode']
neighborhoods_venues_sorted['borough'] = toronto_grouped['borough']
neighborhoods_venues_sorted['nbh'] = toronto_grouped['nbh']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(100, 8)


,postcode,borough,nbh,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",Fast Food Restaurant,Yoga Studio,Drugstore,Discount Store,Distribution Center
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",Construction & Landscaping,Bar,Yoga Studio,Dumpling Restaurant,Dive Bar
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",Intersection,Breakfast Spot,Electronics Store,Restaurant,Medical Center
3,M1G,Scarborough,Woburn,Coffee Shop,Mexican Restaurant,Korean BBQ Restaurant,Distribution Center,Dive Bar
4,M1H,Scarborough,Cedarbrae,Gas Station,Fried Chicken Joint,Bakery,Bank,Athletics & Sports
...,...,...,...,...,...,...,...,...
95,M9N,York,Weston,Park,Yoga Studio,Dumpling Restaurant,Distribution Center,Dive Bar
96,M9P,Etobicoke,Westmount,Pizza Place,Coffee Shop,Sandwich Place,Discount Store,Chinese Restaurant
97,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",Pizza Place,Park,Sandwich Place,Bus Line,Doner Restaurant
98,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",Pizza Place,Grocery Store,Fried Chicken Joint,Sandwich Place,Pharmacy


<h2>Cluster Analysis</h2>
<h4>
Next, I'll work with creating clusters by using the KMeans clustering algorithm.

</h4>

In [40]:
# set number of clusters
kclusters = 4
toronto_grouped_clustering = toronto_grouped.drop(["postcode","borough", "nbh"], axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 3,
       1, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1,
       1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 0, 1, 1, 2, 3, 1, 1, 1, 1])

In [41]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [42]:
neighborhoods_venues_sorted

,Cluster Labels,postcode,borough,nbh,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,1,M1B,Scarborough,"Malvern, Rouge",Fast Food Restaurant,Yoga Studio,Drugstore,Discount Store,Distribution Center
1,2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",Construction & Landscaping,Bar,Yoga Studio,Dumpling Restaurant,Dive Bar
2,1,M1E,Scarborough,"Guildwood, Morningside, West Hill",Intersection,Breakfast Spot,Electronics Store,Restaurant,Medical Center
3,1,M1G,Scarborough,Woburn,Coffee Shop,Mexican Restaurant,Korean BBQ Restaurant,Distribution Center,Dive Bar
4,1,M1H,Scarborough,Cedarbrae,Gas Station,Fried Chicken Joint,Bakery,Bank,Athletics & Sports
...,...,...,...,...,...,...,...,...,...
95,3,M9N,York,Weston,Park,Yoga Studio,Dumpling Restaurant,Distribution Center,Dive Bar
96,1,M9P,Etobicoke,Westmount,Pizza Place,Coffee Shop,Sandwich Place,Discount Store,Chinese Restaurant
97,1,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",Pizza Place,Park,Sandwich Place,Bus Line,Doner Restaurant
98,1,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",Pizza Place,Grocery Store,Fried Chicken Joint,Sandwich Place,Pharmacy


In [43]:
# Merge the venues with the df3, so i can have lat and long back!
tdnew = td
tdnew = td.drop(['postcode', 'borough'], axis = 1)
tdnew

,nbh,lat,lon
0,"Malvern, Rouge",43.806686,-79.194353
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,Woburn,43.770992,-79.216917
4,Cedarbrae,43.773136,-79.239476
...,...,...,...
98,Weston,43.706876,-79.518188
99,Westmount,43.696319,-79.532242
100,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [44]:
tdnew

,nbh,lat,lon
0,"Malvern, Rouge",43.806686,-79.194353
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,Woburn,43.770992,-79.216917
4,Cedarbrae,43.773136,-79.239476
...,...,...,...
98,Weston,43.706876,-79.518188
99,Westmount,43.696319,-79.532242
100,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [45]:

venues_merged = tdnew
venues_merged = venues_merged.join(neighborhoods_venues_sorted.set_index('nbh'), on='nbh')
venues_merged.dropna(inplace=True)

In [46]:
venues_merged

,nbh,lat,lon,Cluster Labels,postcode,borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Malvern, Rouge",43.806686,-79.194353,1.0,M1B,Scarborough,Fast Food Restaurant,Yoga Studio,Drugstore,Discount Store,Distribution Center
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2.0,M1C,Scarborough,Construction & Landscaping,Bar,Yoga Studio,Dumpling Restaurant,Dive Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,M1E,Scarborough,Intersection,Breakfast Spot,Electronics Store,Restaurant,Medical Center
3,Woburn,43.770992,-79.216917,1.0,M1G,Scarborough,Coffee Shop,Mexican Restaurant,Korean BBQ Restaurant,Distribution Center,Dive Bar
4,Cedarbrae,43.773136,-79.239476,1.0,M1H,Scarborough,Gas Station,Fried Chicken Joint,Bakery,Bank,Athletics & Sports
...,...,...,...,...,...,...,...,...,...,...,...
98,Weston,43.706876,-79.518188,3.0,M9N,York,Park,Yoga Studio,Dumpling Restaurant,Distribution Center,Dive Bar
99,Westmount,43.696319,-79.532242,1.0,M9P,Etobicoke,Pizza Place,Coffee Shop,Sandwich Place,Discount Store,Chinese Restaurant
100,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,1.0,M9R,Etobicoke,Pizza Place,Park,Sandwich Place,Bus Line,Doner Restaurant
101,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,1.0,M9V,Etobicoke,Pizza Place,Grocery Store,Fried Chicken Joint,Sandwich Place,Pharmacy


In [47]:
# Convert the clusters to an integer
venues_merged['Cluster Labels'] = venues_merged['Cluster Labels'].astype(int)
venues_merged



,nbh,lat,lon,Cluster Labels,postcode,borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Malvern, Rouge",43.806686,-79.194353,1,M1B,Scarborough,Fast Food Restaurant,Yoga Studio,Drugstore,Discount Store,Distribution Center
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2,M1C,Scarborough,Construction & Landscaping,Bar,Yoga Studio,Dumpling Restaurant,Dive Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,M1E,Scarborough,Intersection,Breakfast Spot,Electronics Store,Restaurant,Medical Center
3,Woburn,43.770992,-79.216917,1,M1G,Scarborough,Coffee Shop,Mexican Restaurant,Korean BBQ Restaurant,Distribution Center,Dive Bar
4,Cedarbrae,43.773136,-79.239476,1,M1H,Scarborough,Gas Station,Fried Chicken Joint,Bakery,Bank,Athletics & Sports
...,...,...,...,...,...,...,...,...,...,...,...
98,Weston,43.706876,-79.518188,3,M9N,York,Park,Yoga Studio,Dumpling Restaurant,Distribution Center,Dive Bar
99,Westmount,43.696319,-79.532242,1,M9P,Etobicoke,Pizza Place,Coffee Shop,Sandwich Place,Discount Store,Chinese Restaurant
100,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,1,M9R,Etobicoke,Pizza Place,Park,Sandwich Place,Bus Line,Doner Restaurant
101,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,1,M9V,Etobicoke,Pizza Place,Grocery Store,Fried Chicken Joint,Sandwich Place,Pharmacy


In [48]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(venues_merged['lat'], venues_merged['lon'], 
                                  venues_merged['nbh'], venues_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


<h4>
This concludes the Week 3 assignment. 
    
    Summary
    
    * In part 1, I created a data frame to include postal code, borough, and neighborhood. The last cell of Part I includes the .shape() function to show the # of rows and columns in the data frame.
    * In part 2, I used the CSV file method to grab the latitude and longitude for each of the neighborhoods, and appended that information to the data frame.
    * In part 3, I explored the Foursquare API to get various venues in the different neighborhoods, and developed a clustering
    approach to showcase the use of clustering with geographic information.

</h4>
Thank you!

Rich Huebner